In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, MarianTokenizer, MarianMTModel

c:\Python3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("./models/opus-mt-tc-big-en-ko")
tokenizer = AutoTokenizer.from_pretrained("./models/opus-mt-tc-big-en-ko")

In [4]:
from datasets import load_dataset

# Tải tập dữ liệu opus-100 với cặp ngôn ngữ tiếng Anh - tiếng Hàn
dataset = load_dataset("Helsinki-NLP/opus-100", "en-ko")


c:\Python3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Phucs\.cache\huggingface\hub\datasets--Helsinki-NLP--opus-100. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 2000/2000 [00:00<00:00, 128884.98 examples/s]


In [12]:
# In ra một vài mẫu trong tập dữ liệu
print(dataset['train'][1])  # Dữ liệu mẫu đầu tiên trong tập huấn luyện
print(dataset)


{'translation': {'en': "I ain't fishing' 'em out.", 'ko': '그거 꺼내려다가는'}}
DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [7]:
def preprocess_function(examples):
    # Lấy câu tiếng Anh từ cột 'en' và câu tiếng Hàn từ cột 'ko'
    inputs = [example['en'] for example in examples['translation']]
    targets = [example['ko'] for example in examples['translation']]
    
    # Mã hóa câu tiếng Anh (inputs)
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    # Mã hóa câu tiếng Hàn (targets) cho nhãn (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    # Thêm nhãn (labels) vào dữ liệu đầu ra
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs


In [8]:
# Áp dụng hàm xử lý dữ liệu cho toàn bộ tập train và validation
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]c:\Python3\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2000/2000 [00:00<00:00, 6332.25 examples/s]


In [13]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Thiết lập các tham số huấn luyện
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
)

# Khởi tạo Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    tokenizer=tokenizer,
)

# Bắt đầu huấn luyện
trainer.train()


c:\Python3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 6/187500 [01:30<750:33:28, 14.41s/it]

KeyboardInterrupt: 